# 提前早停正则化 (Early Stopping)

---

## 📚 学习目标

通过本教程，你将学会：
- 理解早停法的原理
- 掌握早停作为正则化技术的作用
- 实现手动早停和使用回调
- 理解验证集在早停中的作用

## 📋 前置知识

- 过拟合概念
- 梯度下降
- 训练/验证/测试集划分

## ⏱️ 预计时间

25-30分钟

---

## 📖 第1部分：理论背景

### 什么是早停？

**早停（Early Stopping）** 是一种隐式正则化技术：在验证误差开始增加时停止训练，而不是等到训练完成。

### 为什么早停有效？

随着训练进行：
1. **初期**：训练误差和验证误差都下降
2. **中期**：验证误差开始趋于平稳或上升
3. **后期**：训练误差继续下降，验证误差增加（过拟合）

早停的目标是在"拐点"处停止训练。

### 早停 vs 其他正则化

| 方法 | 机制 | 优点 |
|------|------|------|
| L1/L2正则化 | 惩罚大参数 | 数学性质好 |
| Dropout | 随机丢弃神经元 | 适合深度网络 |
| 早停 | 限制训练时间 | 简单，无需调超参 |

> 💡 **关键洞察**: 早停可以看作是对参数空间的隐式正则化——参数没有足够时间走向极端值。

---

## 💻 第2部分：代码实现

### 步骤1: 准备工作

In [ ]:
# ============================================================
# 导入库
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.base import clone

# ============================================================
# 配置
# ============================================================

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 配置完成")

### 步骤2: 生成数据

In [ ]:
# ============================================================
# 生成二次关系数据
# ============================================================

n_samples = 100

X = 6 * np.random.rand(n_samples, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(n_samples, 1)

# 划分训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED
)

print(f"✓ 数据生成完成")
print(f"  训练集: {len(X_train)} 样本")
print(f"  验证集: {len(X_val)} 样本")

### 步骤3: 特征预处理

In [ ]:
# ============================================================
# 创建高次多项式特征（容易过拟合）
# ============================================================

POLY_DEGREE = 20  # 使用20次多项式，容易过拟合

# 预处理管道：多项式特征 + 标准化
preprocessor = Pipeline([
    ('poly', PolynomialFeatures(degree=POLY_DEGREE, include_bias=False)),
    ('scaler', StandardScaler())
])

# 转换数据
X_train_poly = preprocessor.fit_transform(X_train)
X_val_poly = preprocessor.transform(X_val)

print(f"✓ 特征转换完成")
print(f"  多项式次数: {POLY_DEGREE}")
print(f"  特征数量: {X_train_poly.shape[1]}")

### 步骤4: 实现早停

In [ ]:
# ============================================================
# 手动实现早停
# ============================================================

# 创建SGD回归器
# warm_start=True: 允许在fit之间保持模型状态
# max_iter=1: 每次fit只进行一次epoch
sgd_reg = SGDRegressor(
    learning_rate='constant',
    eta0=0.001,
    max_iter=1,
    warm_start=True,     # 关键：保持上一次训练的状态
    penalty=None,        # 无正则化（依靠早停）
    random_state=RANDOM_SEED,
    tol=None             # 禁用自动停止
)

# 早停参数
n_epochs = 500
patience = 20  # 容忍验证误差不下降的epoch数

# 记录历史
train_errors = []
val_errors = []

# 最佳模型追踪
minimum_val_error = float('inf')
best_epoch = None
best_model = None
epochs_without_improvement = 0

print("开始训练（带早停）...")

for epoch in range(n_epochs):
    # 训练一个epoch
    sgd_reg.fit(X_train_poly, y_train.ravel())
    
    # 计算误差
    y_train_pred = sgd_reg.predict(X_train_poly)
    y_val_pred = sgd_reg.predict(X_val_poly)
    
    train_error = mean_squared_error(y_train, y_train_pred)
    val_error = mean_squared_error(y_val, y_val_pred)
    
    train_errors.append(train_error)
    val_errors.append(val_error)
    
    # 检查是否是最佳模型
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = clone(sgd_reg)
        best_model.coef_ = sgd_reg.coef_.copy()
        best_model.intercept_ = sgd_reg.intercept_.copy()
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
    
    # 早停检查
    if epochs_without_improvement >= patience:
        print(f"\n早停触发！在epoch {epoch}停止")
        break

print(f"\n训练完成:")
print(f"  最佳epoch: {best_epoch}")
print(f"  最小验证MSE: {minimum_val_error:.6f}")
print(f"  总训练epochs: {len(train_errors)}")

---

## 📊 第3部分：可视化早停效果

In [ ]:
# ============================================================
# 可视化训练过程
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左图：误差曲线
axes[0].plot(train_errors, 'b-', linewidth=2, label='训练误差')
axes[0].plot(val_errors, 'r-', linewidth=2, label='验证误差')
axes[0].axvline(x=best_epoch, color='green', linestyle='--', linewidth=2,
               label=f'最佳停止点 (epoch={best_epoch})')
axes[0].scatter([best_epoch], [minimum_val_error], c='green', s=100, zorder=5)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE')
axes[0].set_title('训练过程中的误差变化', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 右图：拟合曲线对比
X_line = np.linspace(-3, 3, 100).reshape(-1, 1)
X_line_poly = preprocessor.transform(X_line)

# 最终模型预测
y_final = sgd_reg.predict(X_line_poly)

# 最佳模型预测
y_best = best_model.predict(X_line_poly)

# 真实曲线
y_true = 0.5 * X_line**2 + X_line + 2

axes[1].scatter(X_train, y_train, c='blue', alpha=0.5, s=30, label='训练数据')
axes[1].scatter(X_val, y_val, c='red', alpha=0.5, s=30, label='验证数据')
axes[1].plot(X_line, y_true, 'g--', linewidth=2, label='真实曲线')
axes[1].plot(X_line, y_best, 'purple', linewidth=2, label=f'早停模型 (epoch={best_epoch})')
axes[1].plot(X_line, y_final, 'orange', linewidth=2, alpha=0.7, label=f'最终模型 (epoch={len(train_errors)-1})')
axes[1].set_xlabel('X')
axes[1].set_ylabel('y')
axes[1].set_title('早停模型 vs 最终模型', fontweight='bold')
axes[1].set_ylim(-5, 15)
axes[1].legend(loc='upper left')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 🔧 第4部分：早停的不同实现方式

In [ ]:
# ============================================================
# 早停类封装
# ============================================================

class EarlyStopping:
    """
    早停类
    
    参数:
        patience: 容忍验证损失不下降的epoch数
        min_delta: 认为有改善的最小变化量
        restore_best_weights: 是否恢复最佳权重
    """
    def __init__(self, patience=10, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_score = None
        self.counter = 0
        self.best_weights = None
        
    def __call__(self, val_score, model):
        """
        检查是否应该停止
        
        返回:
            True: 应该停止训练
            False: 继续训练
        """
        if self.best_score is None:
            self.best_score = val_score
            self._save_weights(model)
            return False
            
        if val_score < self.best_score - self.min_delta:
            # 有改善
            self.best_score = val_score
            self._save_weights(model)
            self.counter = 0
        else:
            # 无改善
            self.counter += 1
            if self.counter >= self.patience:
                if self.restore_best_weights:
                    self._restore_weights(model)
                return True
        return False
    
    def _save_weights(self, model):
        self.best_weights = {
            'coef': model.coef_.copy(),
            'intercept': model.intercept_.copy()
        }
    
    def _restore_weights(self, model):
        if self.best_weights:
            model.coef_ = self.best_weights['coef']
            model.intercept_ = self.best_weights['intercept']

print("✓ EarlyStopping类定义完成")

In [ ]:
# ============================================================
# 使用EarlyStopping类
# ============================================================

# 重新创建模型
sgd_reg2 = SGDRegressor(
    learning_rate='constant',
    eta0=0.001,
    max_iter=1,
    warm_start=True,
    penalty=None,
    random_state=RANDOM_SEED,
    tol=None
)

early_stopping = EarlyStopping(patience=20, min_delta=0.001)

train_errors2 = []
val_errors2 = []

for epoch in range(500):
    sgd_reg2.fit(X_train_poly, y_train.ravel())
    
    train_error = mean_squared_error(y_train, sgd_reg2.predict(X_train_poly))
    val_error = mean_squared_error(y_val, sgd_reg2.predict(X_val_poly))
    
    train_errors2.append(train_error)
    val_errors2.append(val_error)
    
    if early_stopping(val_error, sgd_reg2):
        print(f"早停触发在epoch {epoch}")
        print(f"最佳验证MSE: {early_stopping.best_score:.6f}")
        break

print(f"\n使用EarlyStopping类训练完成")

---

## 📝 总结

### 关键要点

1. ✅ **早停**是一种隐式正则化技术
2. ✅ 在验证误差开始增加时停止训练
3. ✅ **patience**参数控制容忍度
4. ✅ 通常要保存并恢复最佳模型权重

### 早停的优缺点

| 优点 | 缺点 |
|------|------|
| 简单有效 | 需要验证集 |
| 无需调整超参数 | 可能过早停止 |
| 节省训练时间 | 不能与其他正则化完美结合 |

### 最佳实践

1. 使用适当的patience值（通常10-20）
2. 始终保存最佳模型
3. 可以与其他正则化技术结合使用
4. 监控训练曲线以调整超参数

### 下一步学习

- 📘 深度学习中的Callbacks
- 📙 模型选择和交叉验证
- 📕 超参数调优

## 📚 参考资料

- 《机器学习实战》第4章
- [TensorFlow: EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping)